# Smallpond

In this Notebook we will be focusing in testing the current environment and getting to know the tools that are involved

## Imports

In [1]:
import ray
from minio import Minio
import os
import duckdb
import pandas as pd
import pyarrow.parquet as pq
from smallpond import init as smallpond_init

## Testing pandas and duckdb
In the following code you will create a simple pandas dataframe and then query it with duckdb and SQL

In [2]:
df = pd.DataFrame({"value": [10, 20, 30, 40, 50]})
result = duckdb.query("SELECT AVG(value) AS avg_value FROM df").to_df()
print(result)

   avg_value
0       30.0


## Testing Ray
You need to initialize Ray before running Smallpond. By sending the parameter address we are making sure to point to the correct Ray cluster.

In [3]:
print("initializing ray>>>>")
ray.init(address="ray://localhost:10001")  # Connects to ray-head which is running inside the cluster
print("Connected to Ray cluster:", ray.cluster_resources())


initializing ray>>>>
Connected to Ray cluster: {'node:172.18.0.3': 1.0, 'CPU': 36.0, 'memory': 15675961346.0, 'object_store_memory': 7028316978.0, 'node:172.18.0.4': 1.0, 'node:__internal_head__': 1.0, 'node:172.18.0.5': 1.0}


In [4]:
@ray.remote
def f(x):
    return x * x

print("Running Ray tasks...")
results = ray.get([f.remote(i) for i in range(4)])
print("Results:", results)

print("Shutting down Ray")
ray.shutdown()

Running Ray tasks...
Results: [0, 1, 4, 9]
Shutting down Ray


If you don't initialize ray then Smallpond will initialize a single node local ray cluster but in that case we are missing the distribution power of Smallpond.

## Testing MinIO

In [5]:
# Connect to MinIO
minio_client = Minio(
    endpoint=os.environ.get("MINIO_ENDPOINT", "http://localhost:9000").replace("http://", ""),
    access_key=os.environ.get("MINIO_ACCESS_KEY", "minioadmin"),
    secret_key=os.environ.get("MINIO_SECRET_KEY", "minioadmin"),
    secure=False,
)

# Create a bucket if it doesn’t exist
bucket_name = "my-data"
if not minio_client.bucket_exists(bucket_name):
    minio_client.make_bucket(bucket_name)
    print(f"Created bucket: {bucket_name}")
else:
    print(f"Bucket already exists: {bucket_name}")

object_name = "data/sample.parquet"
local_file_path = "sample.parquet"  # "/Full/ path /to /your /file/data/sample.parquet"
# Upload the file
minio_client.fput_object(
            bucket_name,
            object_name,
            local_file_path,
)
print(f"File '{local_file_path}' uploaded as '{object_name}' to bucket '{bucket_name}'.")

Bucket already exists: my-data
File 'sample.parquet' uploaded as 'data/sample.parquet' to bucket 'my-data'.


### Let's go to the MinIO dashboard

## Testing Smallpond
In your virtual environment make sure to have the correct MinIO credentials running the following line:

In [3]:
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
os.environ["AWS_REGION"] = "us-east-1"
os.environ["AWS_ENDPOINT_URL"] = "http://localhost:9000"

This way we can pull data directly like we do from an S3 bucket with the notation: s3://

In [4]:
# Get's the data from the parquet file (in your bucket) using PyArrow
a_table = pq.read_table("s3://my-data/data/sample.parquet")

Initialize Ray and Smallpond

In [5]:
# Initialize Ray
ray.init(address="ray://localhost:10001")

# Smallpond is pointing the ray head at ray://localhost:10001
sp = smallpond_init(job_name="ProcessingTest", ray_address="ray://localhost:10001", data_root="data", num_executors=2, bind_numa_node=False, executor_resources={"CPU": 4,"memory": 8,"object_store_memory": 2*1024})

df = sp.from_arrow(a_table)
print(df.take(10))




SIGTERM handler is not set because current thread is not the main thread.


[2025-10-23 11:16:42.759586] [ZG-KPVW7DJYWV] [MainProcess(47988)] [task.py:414] INFO initialized logging to file: /Users/valeryca/PycharmProjects/lightweight-analytics-duckdb-smallpond-oct2025/workshop/notebooks/data/2025-10-23-11-16-42.16c098be-78b2-4ea9-bcc7-2c8d55556e39/log/ZG-KPVW7DJYWV.log
[2025-10-23 11:16:42.763226] [ZG-KPVW7DJYWV] [MainProcess(47988)] [task.py:426] INFO initialized user logging to file: /Users/valeryca/PycharmProjects/lightweight-analytics-duckdb-smallpond-oct2025/workshop/notebooks/data/2025-10-23-11-16-42.16c098be-78b2-4ea9-bcc7-2c8d55556e39/log/ZG-KPVW7DJYWV-user.log
[2025-10-23 11:16:42.763545] [ZG-KPVW7DJYWV] [MainProcess(47988)] [task.py:371] INFO set environment variable 'LD_LIBRARY_PATH': '/Users/valeryca/PycharmProjects/lightweight-analytics-duckdb-smallpond-oct2025/.venv/lib:'
[2025-10-23 11:16:42.763638] [ZG-KPVW7DJYWV] [MainProcess(47988)] [task.py:373] INFO RANK='' LD_LIBRARY_PATH='/Users/valeryca/PycharmProjects/lightweight-analytics-duckdb-smallp

I0000 00:00:1761239802.749136  770857 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1761239802.764327  770857 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1761239802.770149  770857 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1761239802.774529  770857 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


RayTaskError(FileNotFoundError): [36mray::000001.DataSourceTask()[39m (pid=465, ip=172.18.0.3)
  File "/Users/valeryca/PycharmProjects/lightweight-analytics-duckdb-smallpond-oct2025/.venv/lib/python3.11/site-packages/smallpond/execution/task.py", line 1152, in exec_task
  File "/home/ray/anaconda3/lib/python3.11/pathlib.py", line 1108, in touch
    fd = os.open(self, flags, mode)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/Users/valeryca/PycharmProjects/lightweight-analytics-duckdb-smallpond-oct2025/workshop/notebooks/data/2025-10-23-11-16-42.16c098be-78b2-4ea9-bcc7-2c8d55556e39/staging/started_tasks/000001.DataSourceTask-000001.0'

I0000 00:00:1761239862.798041  774232 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
2025-10-23 11:17:42,810	WARNING api.py:293 -- Timeline will include events from other clients using this server.


[2025-10-23 11:17:42.809611] [ZG-KPVW7DJYWV] [MainProcess(47988)] [session.py:264] WARNING graphviz is not installed, skipping graph dump
[2025-10-23 11:17:43.843087] [ZG-KPVW7DJYWV] [MainProcess(47988)] [session.py:315] INFO progress: 1/1 tasks (100.0%)


I0000 00:00:1761239923.850388  774232 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
2025-10-23 11:18:43,857	WARNING api.py:293 -- Timeline will include events from other clients using this server.


[2025-10-23 11:18:43.857168] [ZG-KPVW7DJYWV] [MainProcess(47988)] [session.py:264] WARNING graphviz is not installed, skipping graph dump
[2025-10-23 11:18:44.878190] [ZG-KPVW7DJYWV] [MainProcess(47988)] [session.py:315] INFO progress: 1/1 tasks (100.0%)


I0000 00:00:1761239984.880564  774232 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
2025-10-23 11:19:44,887	WARNING api.py:293 -- Timeline will include events from other clients using this server.


[2025-10-23 11:19:44.887216] [ZG-KPVW7DJYWV] [MainProcess(47988)] [session.py:264] WARNING graphviz is not installed, skipping graph dump
[2025-10-23 11:19:45.905800] [ZG-KPVW7DJYWV] [MainProcess(47988)] [session.py:315] INFO progress: 1/1 tasks (100.0%)


In [ ]:
# Shuts down Ray
ray.shutdown()

### Now let's check the Smallpond Dashboard